In [10]:
!conda list

# packages in environment at /home/niell_lab/anaconda3/envs/tmp_ephys:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
asttokens                 2.0.5              pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
blosc                     1.21.0               h4ff587b_1  
bottleneck                1.3.5            py38h7deecbd_0  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2023.01.10           h06a4308_0  
certifi                   2022.12.7        py38h06a4308_0  
comm                      0.1.2            py38h06a4308_0  
debugpy                   1.5.1            py38h295c915_0  
decorator                 5.1.1              pyhd3eb1b0_0  
entrypoints               0.4              py38h06a4308_0  
executing           

In [6]:
import pandas as pd
import xarray as xr
import os
import fnmatch
import numpy as np
import time
from tqdm import tqdm

In [7]:
def show_xr_objs(df):
    ret = []
    for col, ser in df.iteritems():
        if type(ser.iloc[0]) == xr.core.dataarray.DataArray:
            ret.append(col)
    return ret

def replace_xr_obj(df):
    for x in show_xr_objs(df):
        for i, val in df[x].iteritems():
            df.at[i,x] = val.values
    return df

def choose_most_recent(paths):
    
    deltas = np.zeros(len(paths))
    for i, f in enumerate(paths):
        deltas[i] = time.time() - os.path.getmtime(f)
    ind = np.argmin(deltas)
    use_f = paths[ind]
    return use_f

def find(pattern, path, MR=False):
    """ Glob for subdirectories.

    Parameters
    --------
    pattern : str
        str with * for missing sections of characters
    path : str
        path to search, including subdirectories
    
    Returns
    --------
    result : list
        list of files matching pattern.
    """
    # initialize the list as empty
    result = []
    # walk though the path directory, and files
    for root, _, files in os.walk(path): 
        # walk to the file in the directory
        for name in files:
            # if the file matches the filetype append to list
            if fnmatch.fnmatch(name,pattern):
                result.append(os.path.join(root,name))
    
    if MR is True:
        return choose_most_recent(result)
        
    elif MR is False:
        return result
    

def list_subdirs(rootdir, givepath=False):
    """ List subdirectories in a root directory.
    """
    paths = []; names = []
    for item in os.scandir(rootdir):
        if os.path.isdir(item):
            if item.name[0]!='.':
                paths.append(item.path)
                names.append(item.name)
    if givepath:
        return paths
    elif not givepath:
        return names

In [8]:
recording_directories = [
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/041422/J599RT', # 1
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/041422/J593LT', # 2
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/041822/J594NC', #3
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/042022/J604RT', # 5
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/042122/J604RT', # 6
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/042222/J594RT', # 7
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/042322/J559NC', # 8
    # '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/050522/G6CK5BRN', # 10
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/050622/J606NC', # 11
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/060822/J611RN', #15
    '/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/060922/J611RN' # 16
]

name_opts = {
    'hf1_revchecker500ms': 'Rc5',
    'hf2_sparsenoiseflash500msISI': 'Sn5is',
    'hf5_sparsenoiseflash500ms': 'Sn5is',
    'hf2_sparseflash': 'Sn',
    'hf2_sparsenoiseflash': 'Sn',
    'hf5_sparseflash': 'Sn',
    'hf4_sparsenoiseflashRAND': 'SnRdm',
    'hf3_sparsenoiseflashRAND': 'SnRdm',
    'hf1_revchecker': 'Rc1',
    'hf2_sparseflashgray': 'Sn5is'
}

In [9]:
for recpath in recording_directories:

    subdirs = list_subdirs(recpath, givepath=True)
    subdirs = [p for p in subdirs if any(s in p for s in list(name_opts.keys()))]

    stim_df_list = []

    for sd in tqdm(subdirs):
        full_stim_label = os.path.split(sd)[1]
        h5_path = find('*ephys_props.h5', sd, MR=True)
        # print(h5_path)
        df = pd.read_hdf(h5_path)
        
        bad = show_xr_objs(df)
        
        if len(bad)>0:
            # print(h5_path, bad)
        
            df = replace_xr_obj(df)
        
            name, ext = os.path.splitext(h5_path)
            savepath = name+'_dropXR'+ext
            print(savepath)
        
            # df.to_hdf(savepath)
                
        

 33%|████████████████████████████████████▎                                                                        | 1/3 [00:00<00:01,  1.41it/s]


AttributeError: Can't get attribute 'PandasIndexingAdapter' on <module 'xarray.core.indexing' from '/home/niell_lab/anaconda3/envs/tmp_ephys/lib/python3.8/site-packages/xarray/core/indexing.py'>

In [ ]:
# name, ext = os.path.splitext('/home/niell_lab/Mounts/Goeppert/nlab-nas/freely_moving_ephys/ephys_recordings/041422/J599RT/hf1_revchecker500ms/041422_J599RT_control_Rig2_hf1_revchecker500ms_ephys_props.h5')

In [ ]:
# name+'_dropXR'+ext